In [110]:
# 엑셀파일 읽기
import pandas as pd

In [111]:
%pip install pandas openpyxl xlrd

Note: you may need to restart the kernel to use updated packages.


In [112]:
# pd.read_excel('./lotto_1193.xls',engine='xlrd')
table = pd.read_html('./lotto_1193.xls')   # 확장자가  xls 이지만 내부에  html형태로 되어 있는 테이블
lotto_df = table[1].loc[2:, 13:]
lotto_df = lotto_df[::-1]  # 순서를 반대로
lotto_df = lotto_df.reset_index(drop=True)  # 인덱스 정렬
lotto_df.tail()

,13,14,15,16,17,18,19
588,9,19,29,35,37,38,31
589,7,9,19,23,26,45,33
590,1,4,11,12,20,41,2
591,10,16,23,36,39,40,11
592,6,9,16,19,24,28,17


In [113]:
# 과거 2개의 데이터가 다음 차수를 예측하는 구조로 데이터 셋을 생성   x,y가 있어야 하기 때문에
# 0:2 2
# 1:3 3
# 2:4 4 
# ....
# ?:len(x)

In [114]:
# 데이터셋 생성 완료
X, y = [],[]
period = 5
for i in range(len(lotto_df)-period):
    X.append(lotto_df.iloc[ i : i+period ])
    y.append(lotto_df.iloc[i+period])  
y[-1]    

13     6
14     9
15    16
16    19
17    24
18    28
19    17
Name: 592, dtype: object

In [115]:
import numpy as np
x_array = []
for x_sample in X:
    x_array.append(x_sample.values.flatten().tolist())
y_array = []
for y_sample in y:
    y_array.append(y_sample.values.flatten().tolist())

x_array = np.array(x_array)
y_array = np.array(y_array)

In [116]:
#  데이터를 분할....  80% 학습  20% 평가
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [117]:
from sklearn.model_selection import train_test_split

In [118]:
y_array

array([['1', '5', '6', ..., '20', '39', '22'],
       ['8', '14', '23', ..., '38', '39', '13'],
       ['4', '8', '18', ..., '39', '44', '41'],
       ...,
       ['1', '4', '11', ..., '20', '41', '2'],
       ['10', '16', '23', ..., '39', '40', '11'],
       ['6', '9', '16', ..., '24', '28', '17']],
      shape=(588, 7), dtype='<U2')

In [119]:
x_train,x_test,y_train,y_test =  train_test_split(x_array,y_array,test_size=0.2,random_state=42)


In [120]:
x_train[0]

array(['2', '4', '5', '17', '27', '32', '43', '11', '24', '32', '33',
       '35', '40', '13', '6', '16', '37', '38', '41', '45', '18', '5',
       '10', '13', '27', '37', '41', '4', '2', '11', '17', '18', '21',
       '27', '6'], dtype='<U2')

In [121]:
# 머신러닝  랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
models = []
for position in range(7):
    y_train_pos =  y_train[:,position]
    y_test_pos =  y_test[:,position]
    # model  생성
    model = RandomForestClassifier(max_depth=10, random_state=42)
    # model  학습
    model.fit(x_train,y_train_pos)

    # 평가
    train_score = model.score(x_train,y_train_pos)
    test_score = model.score(x_test,y_test_pos)
    print(f'{position+1}번번호의 학습 정확도 : {train_score}')
    print(f'{position+1}번번호의 테스트 정확도 : {test_score}')
    print()
    models.append(model)


1번번호의 학습 정확도 : 1.0
1번번호의 테스트 정확도 : 0.1271186440677966

2번번호의 학습 정확도 : 1.0
2번번호의 테스트 정확도 : 0.05084745762711865

3번번호의 학습 정확도 : 0.997872340425532
3번번호의 테스트 정확도 : 0.059322033898305086

4번번호의 학습 정확도 : 1.0
4번번호의 테스트 정확도 : 0.025423728813559324

5번번호의 학습 정확도 : 0.997872340425532
5번번호의 테스트 정확도 : 0.09322033898305085

6번번호의 학습 정확도 : 1.0
6번번호의 테스트 정확도 : 0.11864406779661017

7번번호의 학습 정확도 : 1.0
7번번호의 테스트 정확도 : 0.01694915254237288



In [122]:
# 가장최근의 5회차의 번호로 예측
temp_x = lotto_df.iloc[-period:].values.flatten().reshape(1,-1)
for idx, model in enumerate(models):
    pred = model.predict(temp_x)[0]
    print(f'{idx+1}번 : {pred}')

1번 : 3
2번 : 6
3번 : 16
4번 : 31
5번 : 37
6번 : 44
7번 : 4
